In [1]:
import alpaca_trade_api as tradeapi
# base_url = 'https://paper-api.alpaca.markets'
# api_key = 'PKHBF2CR3BO32XBYVC0Y'
# secret_key = 'ZbKr2fylL6mtbbDtK9CeiN61iZJ5qXqLFmCAT2pi'

# api = tradeapi.REST(api_key, secret_key,base_url)
api_key = 'AKHME2093RBNAQYHPUAV'
base_url = 'https://api.alpaca.markets'
secret_key = 'bEWXQSpd8FHvzZgH5LFZGv0s1iPYJjOYHF5nX3LI'
api = tradeapi.REST(api_key, secret_key)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import json
import pprint as pprint
import pandas as pd
from joblib import dump, load
import requests
import math
model = load('V3Model.joblib')
X_scaler = load('V3X_scaler.joblib')
y_scaler = load('V3y_scaler.joblib')
import alpaca_trade_api as tradeapi
api_key = 'AKHME2093RBNAQYHPUAV'
base_url = 'https://api.alpaca.markets'
secret_key = 'bEWXQSpd8FHvzZgH5LFZGv0s1iPYJjOYHF5nX3LI'
api = tradeapi.REST(api_key, secret_key)
from twilio.rest import Client
account_sid = 'AC4ab3ece33d5bb0b0a6d617086b40ae98'
auth_token = '285a6c2756f288274702c9e3de20003c'
client = Client(account_sid, auth_token)
import os
import pandas as pd
account = api.get_account()
df = pd.read_csv('./V3List.csv')
tckr = df['Tckr'].tolist()
tckr.remove('SPRT')
pp = math.floor((round(float(account.cash),1)/3)*.97)
pp

In [ ]:
import pandas as pd
import json
import requests
from pprint import pprint
import alpaca_trade_api as tradeapi
api_key = 'AKHME2093RBNAQYHPUAV'
base_url = 'https://api.alpaca.markets'
secret_key = 'bEWXQSpd8FHvzZgH5LFZGv0s1iPYJjOYHF5nX3LI'
api = tradeapi.REST(api_key, secret_key)
price_info= []
count = 0
for i in tckr:
    df = pd.read_csv('./V3List.csv')
    tckr = df['Tckr'].tolist()
    try:
        barset = api.get_barset(i, 'day', limit=2)
        
        openingY = barset[i][0].o
        closingY = barset[i][0].c
        highY = barset[i][0].h
        lowY = barset[i][0].l
        openingT = barset[i][1].o
        closingT = barset[i][1].c
        highT = barset[i][1].h
        lowT = barset[i][1].l
        info = {'Ticker':f'{i}',
                'openingY':openingY,
                'closingY':closingY,
                'highY':highY,
                'lowY':lowY,
                'openingT':openingT,
                'closingT':closingT,
                'highT':highT,
                'lowT':lowT}
        pprint(i)
        count+=1
        price_info.append(info)
#         pprint(price_info)
        
        

    except:
        
        count+=1

        print(f'result error{i}')
        pass

In [ ]:
df = pd.DataFrame(price_info)
df

In [ ]:
X = df[['openingY', 'closingY', 'highY', 'lowY']]
y = df["highT"].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
dump(X_scaler, 'V3X_scaler.joblib') 
dump(y_scaler, 'V3y_scaler.joblib') 
model = LinearRegression()
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
model.fit(X_train_scaled, y_train_scaled)
predictions = model.predict(X_test_scaled)
r2 = model.score(X_test_scaled, y_test_scaled)
r2= round(float(r2),5)
dump(model, 'V3Model.joblib') 
ldf = df[['openingT', 'closingT', 'highT', 'lowT']]
sc_new_in = X_scaler.transform(ldf)
new_out = model.predict(sc_new_in)
new_out = y_scaler.inverse_transform(new_out)
df["next_high"]= new_out
df["change"]= df['next_high']/df["closingT"]*100-100
rdf = df.sort_values(by=['change'], axis=0, ignore_index=True, inplace=False, ascending = False)
rdf = rdf.drop(0, axis=0)
rdf = rdf.reset_index(inplace = False)
purchase_list = rdf.head(3)
purchase_list

In [ ]:
c=0
rdf = rdf.drop(c, axis=0)
rdf = rdf.reset_index(inplace = False)
rdf

In [ ]:
p=0
c = 0
message_list = []
for i in purchase_list.Ticker:
    try:
        if p<3:
#             print(purchase_list)


            api.submit_order(
                symbol=i,
                qty=math.floor(pp/purchase_list.highT[c]),
                side='buy',
                type='market',
                time_in_force='day'
            )
            message_list.append(purchase_list.Ticker[c])
            message_list.append(math.floor(pp/purchase_list.highT[c]))
            message_list.append(purchase_list.change[c])

            p+=1
            c+=1

        else:
            break
    except:
        
        rdf = rdf.drop(c, axis=0)
        rdf = rdf.reset_index(inplace = False)
        purchase_list = rdf.head(3)
        c+=1

        print(purchase_list)
        pass
# message = client.messages\
#                 .create(
#                     body=f"Model accuracy rating of {r2}."
#         f"{message_list[1]} shares of {message_list[0]} projected with {message_list[2]}change. "
#         f"{message_list[4]} shares of {message_list[3]} projected with {message_list[5]}change. "
#         f"{message_list[7]} shares of {message_list[6]} projected with {message_list[8]}change. ",
#                     from_='+19703153376',
#                     to='+13033044215'
#                 )

In [ ]:
X = df[['openingY', 'closingY', 'highY', 'lowY']]
y = df["highT"].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
dump(X_scaler, 'V3X_scaler.joblib') 
dump(y_scaler, 'V3y_scaler.joblib') 
model = LinearRegression()
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
model.fit(X_train_scaled, y_train_scaled)
predictions = model.predict(X_test_scaled)
r2 = model.score(X_test_scaled, y_test_scaled)
dump(model, 'V3Model.joblib') 
ldf = df[['openingT', 'closingT', 'highT', 'lowT']]
sc_new_in = X_scaler.transform(ldf)
new_out = model.predict(sc_new_in)
new_out = y_scaler.inverse_transform(new_out)
df["next_high"]= new_out
df["change"]= df['next_high']/df["closingT"]*100-100
rdf = df.sort_values(by=['change'], axis=0, ignore_index=True, inplace=False, ascending = False)
# rdf = rdf.drop(0, axis=0)
# rdf = rdf.reset_index(inplace = False)
purchase_list = rdf.head(3)
purchase_list
c = 0
message_list = []
for i in purchase_list.Ticker:
    api.submit_order(
    symbol=i,
    qty=math.floor(pp/purchase_list.highT[c]),
    side='buy',
    type='market',
    time_in_force='day'
    )
    message_list.append(purchase_list.Ticker[c])
    message_list.append(math.floor(pp/purchase_list.highT[c]))
    message_list.append(purchase_list.change[c])

    c+=1
message = client.messages \
                .create(
                    body=f"Model accuracy rating of {r2}. "
        f"{message_list[1]} Shares of {message_list[0]} projected with {message_list[2]}change. "
        f"{message_list[4]} Shares of {message_list[3]} projected with {message_list[5]}change. "
        f"{message_list[7]} Shares of {message_list[6]} projected with {message_list[8]}change. ",
                    from_='+19703153376',
                    to='+13033044215'
                )

In [ ]:
positions = api.list_positions()
# positions.pop(0)
positions

In [ ]:
c = 0
positions = api.list_positions()
for i in positions:
    api.submit_order(
    symbol=positions[c].symbol,
    qty=positions[c].qty,
    side='sell',
    type='trailing_stop',
    trail_percent=.8,
    time_in_force='gtc',
    )
    c+=1

In [ ]:
print(message_list)


In [2]:
active_assets = api.list_assets()
active_assets[0]

Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'AMEX',
    'fractionable': False,
    'id': '613cb2a6-c53c-4895-a5a3-e8e4dd7d35ce',
    'marginable': True,
    'name': 'AgeX Therapeutics, Inc.',
    'shortable': False,
    'status': 'active',
    'symbol': 'AGE',
    'tradable': True})

In [3]:
tradeable=[]
count=0
for i in range(len(active_assets)):
    if (active_assets[i].exchange) == ('NASDAQ') or ('NYSE'):
        print(active_assets[i].exchange)
        count+=1
        tradeable.append(active_assets[i].symbol)

AMEX
NYSE
AMEX
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
AMEX
NYSE
OTC
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NYSE
NASDAQ
NASDAQ
ARCA
NYSE
NASDAQ
BATS
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
ARCA
NYSE
NYSE
NYSE
NASDAQ
ARCA
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
BATS
NYSE
NYSE
NASDAQ
BATS
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
AMEX
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
ARCA
NYSE
NYSE
NASDAQ
ARCA
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
AMEX
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
OTC
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
ARCA
NASDAQ
AMEX
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
NYSE
BATS
OTC
NASDAQ
OTC
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
ARCA
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ

NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
AMEX
ARCA
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
ARCA
ARCA
ARCA
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
BATS
NASDAQ
ARCA
BATS
NYSE
ARCA
NASDAQ
BATS
NASDAQ
ARCA
NYSE
ARCA
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
ARCA
ARCA
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
ARCA
NASDAQ
BATS
ARCA
NYSE
NASDAQ
NYSE
NYSE
ARCA
ARCA
NASDAQ
NASDAQ
OTC
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NYSE
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
ARCA
NASDAQ
NASDAQ
ARCA
ARCA
ARCA
NYSE
ARCA
NYSE
AMEX
BATS
NYSE
AMEX
NYSE
NASDAQ
BATS
NYSE
ARCA
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
ARCA
ARCA
NASDAQ
NASDAQ
ARCA
NASDAQ
AMEX
ARCA
NYSE
NASDAQ
ARCA
NYSE
NYSE
ARCA
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ

NYSE
NYSE
NYSE
ARCA
ARCA
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
ARCA
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
ARCA
ARCA
NASDAQ
NYSE
ARCA
NASDAQ
NYSE
OTC
NASDAQ
NYSE
NYSE
NASDAQ
BATS
ARCA
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
ARCA
NYSE
NYSE
AMEX
ARCA
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
ARCA
NYSE
ARCA
NASDAQ
NASDAQ
ARCA
ARCA
NASDAQ
ARCA
NASDAQ
ARCA
NYSE
NASDAQ
OTC
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
ARCA
ARCA
NYSE
NYSE
NASDAQ
NYSE
ARCA
ARCA
NYSE
NASDAQ
NASDAQ
ARCA
NYSE
ARCA
NYSE
NASDAQ
NYSE
NYSE
ARCA
NASDAQ
NASDAQ
NYSE
NASDAQ
BATS
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
ARCA
NASDAQ
BATS
NASDAQ
ARCA
ARCA
NYSE
BATS
NYSE
ARCA
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
OTC
NYSE
NYSE
NYSE
ARCA
NASDAQ
AMEX
NASDAQ
ARCA
NYSE
NASDAQ
NASD

ARCA
NYSE
BATS
BATS
ARCA
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
ARCA
NYSE
ARCA
NASDAQ
NYSE
ARCA
BATS
NYSE
BATS
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
ARCA
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
AMEX
NYSE
NASDAQ
ARCA
NYSE
ARCA
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
ARCA
NASDAQ
ARCA
NASDAQ
NASDAQ
NYSE
NASDAQ
ARCA
NYSE
BATS
ARCA
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
BATS
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
ARCA
OTC
BATS
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
ARCA
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
BATS
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
ARCA
AMEX
NASDAQ
ARCA
NYSE
NASDAQ
BATS
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
BATS
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
BATS
ARCA
NASDAQ
ARCA
NYSE
ARCA
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
ARCA
NYSE
ARCA
NYSE
NASDAQ
BATS
NYSE
NYSE
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
ARCA
NASDAQ
N

NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
ARCA
NYSE
ARCA
NASDAQ
NASDAQ
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
ARCA
ARCA
ARCA
ARCA
ARCA
ARCA
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NYSE
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
BATS
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
BATS
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
ARCA
BATS
NASDAQ
NASDAQ
NASDAQ
NASDAQ
BATS
NASDAQ
NASDAQ
ARCA
ARCA
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
OTC
ARCA
NASDAQ
NASDAQ
ARCA
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
ARCA
AMEX
NYSE
NYSE
ARCA
BATS
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NASDAQ
AMEX
NASDAQ
OTC
NYSE
ARCA
NYSE


NYSE
OTC
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
ARCA
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
BATS
NASDAQ
NYSE
NYSE
NASDAQ
OTC
NASDAQ
NYSE
NASDAQ
ARCA
NASDAQ
NASDAQ
NASDAQ
NYSE
ARCA
NASDAQ
ARCA
OTC
NYSE
NYSE
NASDAQ
NASDAQ
BATS
ARCA
NASDAQ
NASDAQ
NASDAQ
OTC
AMEX
AMEX
NASDAQ
ARCA
NASDAQ
OTC
NASDAQ
NASDAQ
ARCA
NYSE
BATS
NYSE
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
NASDAQ
AMEX
NASDAQ
ARCA
OTC
NYSE
BATS
NASDAQ
AMEX
NYSE
NYSE
OTC
NASDAQ
NASDAQ
NYSE
ARCA
ARCA
NYSE
NYSE
ARCA
NASDAQ
NASDAQ
BATS
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
OTC
NASDAQ
ARCA
NASDAQ
ARCA
ARCA
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
ARCA
NYSE
NYSE
NYSE
BATS
NASDAQ
ARCA
BATS
NYSE
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
BATS
NYSE
ARCA
OTC
ARCA
NASDAQ
ARCA
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
BATS
NYSE
BATS
NASDAQ
ARCA
NASDAQ
ARCA
NYSE
NYSE
OTC
NYSE
NYSE
NASDAQ
NYSE
ARCA
NASDAQ
ARCA
ARCA
BATS
NASDAQ
NYSE
NASDAQ
NASDAQ
AMEX
NYSE
ARCA
ARCA
ARCA
OTC
ARCA
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NASD

ARCA
OTC
NYSE
OTC
NYSE
OTC
NASDAQ
NYSE
NYSE
ARCA
NASDAQ
NYSE
NASDAQ
NYSE
BATS
NYSE
NASDAQ
NYSE
NASDAQ
BATS
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NYSE
ARCA
NYSE
OTC
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
BATS
NYSE
ARCA
NASDAQ
OTC
NYSE
NYSE
BATS
AMEX
NYSE
NASDAQ
NASDAQ
ARCA
AMEX
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
OTC
NYSE
NASDAQ
ARCA
NASDAQ
NYSE
NASDAQ
NASDAQ
ARCA
NASDAQ
BATS
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
ARCA
ARCA
NASDAQ
ARCA
NYSE
ARCA
ARCA
BATS
NYSE
NASDAQ
NYSE
NASDAQ
BATS
NASDAQ
ARCA
NASDAQ
NASDAQ
ARCA
ARCA
NASDAQ
NASDAQ
ARCA
NASDAQ
ARCA
ARCA
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
ARCA
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
OTC
ARCA
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
BATS
ARCA
ARCA
ARCA
OTC
ARCA
NASDAQ
AMEX
NYSE
ARCA
AMEX
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
ARCA
OTC
NASDAQ
ARCA
NYSE
ARCA
BATS
NYSE
ARCA
NYSE
NASDAQ
ARCA
NASDAQ
ARCA
NASDAQ
BATS
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
ARCA
NASDAQ
OTC
NASDA

In [4]:
tradeable=[]
count=0
for i in range(len(active_assets)):
    if (active_assets[i].tradable) == True and ((active_assets[i].exchange) == 'NASDAQ' or (active_assets[i].exchange) == 'NYSE'):
        print(active_assets[i].exchange)
        count+=1
        tradeable.append(active_assets[i].symbol)

NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASD

NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NA

NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ

NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NYS

NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NYSE
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NASDAQ
NYSE
NASDAQ
NYSE
NASDAQ
NYSE
NYSE
NYSE
NASDAQ
NASDAQ
NASDAQ
NASDAQ
NY

In [10]:
import pandas as pd

treadableDict = {'Tckr': tradeable}
df = pd.DataFrame(treadableDict)
df.to_csv(r'../Machine_learning/Alpacca_NYSE_NASDAQ_List.csv', index = False)


In [ ]:
l = len(tradeable)
l

In [ ]:
df = []

for i in tradeable:
    try:
        barset = api.get_barset(i, 'day', limit=1)
        h = barset[i][0].h
        info = {
        'Tckr' :i,
        'h' :h}
        df.append(info)
        print(i),
    except:
        print(f'error {i}')
        pass

In [ ]:
import pandas as pd

df = pd.DataFrame(df)
df = df.sort_values(by='h', ascending=False)
df.to_csv(r'../Machine_learning/Alpacca_NYSE_NASDAQ_List.csv', index = False)
df = df.head(4000)
stList3 = df['Tckr'].tolist()
df

In [ ]:

df.to_csv(r'../Machine_learning/V3List.csv', index = False)